## CriteriaGroup is another (preferred) way to specify AND and OR conditions in the query: Fetch the female patient cohort with Low back pain OR Type 2 diabetes mellitus with living patients

Setup the imports.

In [ ]:
from pyqe import *

Always start with creating a query object. 

Apart from creating Query object, it does the following that are required for setting up your profile:
- Ask for username and password for login

In [ ]:
# Always begin your script by creating Query object
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive = Query("female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive")

Use get_study_list() method to fetch all available studies that you have access to.

Use set_study() method to select a study that you are interested in (by passing the study ID).

In [ ]:
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive.get_study_list()

# any STUDY_ID from above list
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive.set_study('703c5d8a-a1d9-4d42-a314-5b9aad513390') 

Similar to above step, use get_config_list() method to fetch all available configurations corresponding to the study.

Use set_study_config() method to select a config (by passing the config ID).

In [ ]:
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive.get_config_list()

# any CONFIG_ID from above list
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive.set_study_config('088791fd-cac7-4281-9d61-6f4efd92d2f5') 

Create two Expression objects as described below:
- ComparisonOperator.EQUAL (strictly match the value) and 'Low back pain' (a value that our expression relies on)
- ComparisonOperator.EQUAL (strictly match the value) and 'Type2 Diabetes Mellitus' (a value that our expression relies on)

In [ ]:
exp_equal_low_backpain = Expression(ComparisonOperator.EQUAL, 'Low back pain')
exp_equal_type2_diabetes_mellitus = Expression(ComparisonOperator.EQUAL, 'Type 2 diabetes mellitus')

Create a low backpain condition constraint:
- Create a Constraint object
- Add the exp_equal_low_backpain to the constraint

In [ ]:
constraint_low_back_pain = Constraint()
constraint_low_back_pain.add(exp_equal_low_backpain)

Create a type 2 diabetes condition constraint:
- Create a Constraint object
- Add the exp_equal_type2_diabetes_mellitus to the constraint

In [ ]:
constraint_type2_diabetes_mellitus = Constraint()
constraint_type2_diabetes_mellitus.add(exp_equal_type2_diabetes_mellitus)

Use Interactions class to create a filtercard of type ConditionOccurrence with name 'Type2 Diabetes Mellitus'.

NOTE: Interactions is a parent class which can help to define/instantiate any filter card that is available in the study configuration. 

Go to Miscellaneous to find out more on how to view all filter cards.

Call add_condition_name() method to add the constraint constraint_type2_diabetes_mellitus

In [ ]:
## 7.1 Create two filtercards, filtercard_diabetes_type2 and filtercard_low_backpain
filtercard_diabetes_type2 = Interactions.ConditionOccurrence('Type2 Diabetes Mellitus')
filtercard_diabetes_type2.add_condition_name([constraint_type2_diabetes_mellitus])

Use Interactions class to create a filtercard of type ConditionOccurrence with name 'Low backpain'.

Call add_condition_name() method to add the constraint constraint_low_back_pain.

In [ ]:
filtercard_low_backpain = Interactions.ConditionOccurrence('Low backpain')
filtercard_low_backpain.add_condition_name([constraint_low_back_pain])

Create CriteriaGroup object criteriagroup_type2_diabetes_or_low_backpain with the following values:
- MatchCriteria.ANY condition (ANY of the filter cards' conditions added to the criteriagroup must hold true)
- List of filter cards that the MatchCriteria condition must be applied on

In [ ]:
## 7.2 create CriteriaGroup with OR condition on above filtercards
criteriagroup_type2_diabetes_or_low_backpain = CriteriaGroup(MatchCriteria.ANY, [filtercard_diabetes_type2, filtercard_low_backpain])

Create a gender constraint:
- Create a Constraint object
- Create and add an expression (that EQUALS 'Female') to the Constraint object

In [ ]:
## 7.3 create Patient filtercard with female constraint
constraint_female_gender = Constraint()
constraint_female_gender.add(Expression(ComparisonOperator.EQUAL, 'Female'))

Create a filter card:
- Create a filtercard of type Patient class
- Call add_gender() to add the constraint constraint_female_gender

NOTE: Patient object has multiple methods in order to add constraints, for example, add_county(), add_race(), add_state(). 

Go to Miscellaneous to find out more on how to view all such methods.

In [ ]:
filtercard_female_patient = Person.Patient()
filtercard_female_patient.add_gender([constraint_female_gender])

Create CriteriaGroup object criteriagroup_female_patient with the following values:
- MatchCriteria.ALL condition (ALL of the filter cards' conditions added to the CriteriaGroup must hold true)
- List of filter cards that the MatchCriteria condition must be applied on

Create a filtercard of type Death. Also add the option CardType.EXCLUDED.

NOTE: During the creation of any interaction, an addition parameter may be passed to specify CardType. 

Default value is INCLUDED, otherwise specify CardType.EXCLUDED to exclude results that satisfies the filtercard conditions.

In [ ]:
## 7.4 Create Death filtercard
filtercard_death = Interactions.Death('dead patients', CardType.EXCLUDED)

A change in the previous example will be to create CriteriaGroup object criteriagroup_female_and_not_dead with filtercard_death and filtercard_female_patient with the MatchCriteria.ALL as shown to strictly match both conditions.

In [ ]:
## 7.5 using CriteriaGroup
criteriagroup_female_and_not_dead = CriteriaGroup(MatchCriteria.ALL, [filtercard_female_patient, filtercard_death])


Similar to the previous script, add criteriagroup_type2_diabetes_or_low_backpain CriteriaGroup object to criteriagroup object criteriagroup_female_and_not_dead as shown.

In [ ]:
criteriagroup_type2_diabetes_or_low_backpain = CriteriaGroup(MatchCriteria.ANY, [filtercard_diabetes_type2, filtercard_low_backpain])
criteriagroup_female_and_not_dead.add_exclusive_group(criteriagroup_type2_diabetes_or_low_backpain)


Similar to the previous script, add the criteriagroup object criteriagroup_female_and_not_dead to the query as shown.

In [ ]:
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive.add_criteria_group(criteriagroup_female_and_not_dead)

Fetch the dataframe cohort

In [ ]:
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive_cohort = query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive.get_dataframe_cohort()
query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive_df = Result().download_dataframe(query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive_cohort)
print(f'\n Number of records : {len(query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive_df)}')
print(f'\n Participants: {query_female_patients_with_low_backpain_or_type2_diabetes_mellitus_and_alive_df.head(10)}')